In [1]:
import os.path
from tqdm import tqdm

import numpy as np
import cv2
from SkeletonData.data import *

In [2]:
def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width, 3], dtype=np.uint8)
    img_3.fill(255)

    # add circles
    for i, coord in enumerate(frame):
        x, y = int(width * coord[0]), int(height * coord[1])
        img_3 = cv2.circle(img_3, center=(x, y), radius=3, color=(255, 0, 0), thickness=6)

    for line in connections:
        i, j = line
        st = frame[i, :]
        start_point = (int(width * st[0]), int(height * st[1]))

        en = frame[j, :]
        end_point = (int(width * en[0]), int(height * en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3


def gen_skeletons(frame1, frame2, connections, height, width):
    img_3 = np.zeros([height, width, 3], dtype=np.uint8)
    img_3.fill(255)

    # add circles for the frame-set-1
    for coord in frame1:
        x, y = int(width * coord[0]), int(height * coord[1])
        img_3 = cv2.circle(img_3, center=(x, y), radius=1, color=(255, 0, 0), thickness=6)

    # add circles for the frame-set-2
    for coord in frame2:
        x, y = int(width * coord[0]), int(height * coord[1])
        img_3 = cv2.circle(img_3, center=(x, y), radius=1, color=(0, 255, 0), thickness=6)

    for line in connections:
        i, j = line

        st = frame1[i, :]
        start_point = (int(width * st[0]), int(height * st[1]))
        en = frame1[j, :]
        end_point = (int(width * en[0]), int(height * en[1]))
        img3_ = cv2.line(img_3, start_point, end_point, color=(250, 0, 0), thickness=3)

        st = frame2[i, :]
        start_point = (int(width * st[0]), int(height * st[1]))
        en = frame2[j, :]
        end_point = (int(width * en[0]), int(height * en[1]))
        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 250, 0), thickness=3)

    return img_3


def gen_video(points, save_file, frame_h, frame_w, mapping_list=None):
    # make 3D if points are flatten
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1, 2, 0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)

    for __id, frame in enumerate(points):
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()


def gen_cmp_video(points1, points2, save_file, frame_h, frame_w, mapping_list=None):
    # make 3D if points are flatten
    if len(points1.shape) == 2:
        fts = points1.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points1 = np.transpose(np.array([points1[:, x_cds], points1[:, y_cds], points1[:, z_cds]]), (1, 2, 0))

    if len(points2.shape) == 2:
        fts = points2.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points2 = np.transpose(np.array([points2[:, x_cds], points2[:, y_cds], points2[:, z_cds]]), (1, 2, 0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                             cv2.VideoWriter_fourcc(*'MJPG'),
                             10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    if not mapping_list:
        # add lines
        mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10),
                        (9, 11)]
    for __id, frame_1 in enumerate(points1):
        frame_2 = points2[__id]
        skel_image = gen_skeletons(frame_1, frame_2, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

### Kinetic V2

In [3]:
#mapping_l = [(4,3),(12,10),(10,9),(9,21),(21,5),(5,6),(6,7),(21,2),(21,2),(2,1),(17,1),(1,13),(17,18),(18,19),(13,14),(14,15)]
#mapping_l = [(v1-1,v2-1) for v1,v2 in mapping_l]

### COCO Format

In [4]:
mapping_l = [
        [15, 13], [13, 11], [11, 5],
        [12, 14], [14, 16], [12, 6],
        [3, 1],[1, 2],[1, 0],[0, 2],[2,4],
        [9, 7], [7,5], [5, 6],
        [6, 8], [8, 10],
        ]

In [5]:
import numpy as np
import  os

In [6]:
os.listdir("../data/sequence_data/random_50f")

['0_cls_S001C001P001R001A001_0.npz',
 '0_cls_S001C001P001R001A001_1.npz',
 '0_cls_S001C001P001R002A001_0.npz',
 '0_cls_S001C001P002R001A001_0.npz',
 '0_cls_S001C001P002R002A001_0.npz',
 '0_cls_S001C001P003R001A001_0.npz',
 '0_cls_S001C001P003R001A001_1.npz',
 '0_cls_S001C001P003R002A001_0.npz',
 '0_cls_S001C001P003R002A001_1.npz',
 '0_cls_S001C001P003R002A001_2.npz',
 '0_cls_S001C001P004R001A001_0.npz',
 '0_cls_S001C001P004R001A001_1.npz',
 '0_cls_S001C001P004R002A001_0.npz',
 '0_cls_S001C001P005R001A001_0.npz',
 '0_cls_S001C001P005R002A001_0.npz',
 '0_cls_S001C001P006R001A001_0.npz',
 '0_cls_S001C001P006R002A001_0.npz',
 '0_cls_S001C001P007R001A001_0.npz',
 '0_cls_S001C001P007R002A001_0.npz',
 '0_cls_S001C001P007R002A001_1.npz',
 '0_cls_S001C001P008R001A001_0.npz',
 '0_cls_S001C001P008R002A001_0.npz',
 '0_cls_S001C002P001R001A001_0.npz',
 '0_cls_S001C002P001R001A001_1.npz',
 '0_cls_S001C002P001R002A001_0.npz',
 '0_cls_S001C002P002R001A001_0.npz',
 '0_cls_S001C002P002R002A001_0.npz',
 

In [7]:
file = np.load("../data/sequence_data/random_50f/0_cls_S014C003P019R002A001_0.npz")
coordinates, vid_size = file["coords"], file["video_size"]

In [9]:
gen_video(coordinates,"vizualize.mp4",1000,1000,mapping_list=mapping_l)